In [4]:
# -*- coding: utf-8 -*-
"""
Advanced Graph Editor & Algorithm Visualizer
Algorithms run in a background thread so the window never freezes.

Controls:
  N  - Add Node mode        E  - Add Edge mode
  D  - Toggle Directed      R  - Reset colours
  Ctrl+Z / Ctrl+Y  - Undo / Redo
  ESC - Cancel current action
"""

import pygame
import threading
import queue as q_mod
import math
import heapq
import copy
import os
import time

pygame.init()
clock = pygame.time.Clock()

SCREEN_W, SCREEN_H = 1100, 700
SIDEBAR_W   = 220
CANVAS_X    = SIDEBAR_W
CANVAS_W    = SCREEN_W - SIDEBAR_W
INFOPANEL_H = 110

screen = pygame.display.set_mode((SCREEN_W, SCREEN_H))
pygame.display.set_caption("Graph Editor & Algorithm Visualizer - Advanced")

# ── colours ─────────────────────────────────────────────────────
BG_DARK      = (30,  30,  38)
BG_SIDEBAR   = (22,  22,  30)
BG_CANVAS    = (38,  38,  50)
BG_INFO      = (20,  20,  28)

ACCENT       = (72, 199, 142)
TEXT_MAIN    = (230,230,235)
TEXT_DIM     = (130,130,145)
TEXT_BRIGHT  = (255,255,255)

C_NODE_DEF  = (72, 199, 142)
C_NODE_VIS  = (100,180,255)
C_NODE_ART  = (255,200, 80)
C_NODE_SRC  = (255,130, 60)
C_NODE_DST  = (255, 90, 90)

C_EDGE_DEF  = (90,  90, 110)
C_EDGE_TREE = (72, 199, 142)
C_EDGE_BACK = (255, 90, 90)
C_EDGE_BRDG = (100,180,255)
C_EDGE_MST  = (180, 90,255)
C_EDGE_PATH = (255,200, 80)

NODE_R = 18

# ── fonts ────────────────────────────────────────────────────────
_FONT = os.path.join("/mnt/user-data/uploads", "roboto.ttf")
def _f(sz):
    try:    return pygame.font.Font(_FONT, sz)
    except: return pygame.font.SysFont("arial", sz)

font_big   = _f(22)
font_med   = _f(16)
font_small = _f(13)
font_tiny  = _f(11)

# ── graph state (shared between main thread & algo thread) ───────
nodes       = []   # [[x,y], ...]
edges       = []   # [[a, b, weight], ...]
directed    = False

node_colors = []   # colour per node
edge_colors = []   # colour per edge

undo_stack  = []
redo_stack  = []

# algo thread sends updates via this queue
# items: ("node", idx, color) | ("edge", idx, color) | ("done", msg1, msg2)
update_q = q_mod.Queue()

algo_running = False

# ── UI state ─────────────────────────────────────────────────────
state        = "idle"
algo_pending = None
src_node     = -1
dst_node     = -1
selected_a   = -1
msg_lines    = ["Welcome!  Build your graph with the sidebar, then run an algorithm."]

# ── helpers ──────────────────────────────────────────────────────
def node_center(i):
    return (nodes[i][0], nodes[i][1])

def edist(a, b):
    return round(math.hypot(nodes[a][0]-nodes[b][0], nodes[a][1]-nodes[b][1]))

def edge_exists(a, b):
    for e in edges:
        if e[0]==a and e[1]==b: return True
        if not directed and e[0]==b and e[1]==a: return True
    return False

def get_adj(weighted=False):
    adj = [[] for _ in nodes]
    for e in edges:
        a, b, w = e
        adj[a].append((b, w) if weighted else b)
        if not directed:
            adj[b].append((a, w) if weighted else a)
    return adj

def get_edge_index(a, b):
    for i, e in enumerate(edges):
        if e[0]==a and e[1]==b: return i
        if not directed and e[0]==b and e[1]==a: return i
    return -1

def node_at(mx, my):
    for i, n in enumerate(nodes):
        if math.hypot(mx-n[0], my-n[1]) <= NODE_R+4: return i
    return -1

def edge_at(mx, my):
    for i, e in enumerate(edges):
        ax, ay = node_center(e[0]); bx, by = node_center(e[1])
        dx, dy = bx-ax, by-ay
        if dx==dy==0: continue
        t = max(0, min(1, ((mx-ax)*dx+(my-ay)*dy)/(dx*dx+dy*dy)))
        if math.hypot(mx-(ax+t*dx), my-(ay+t*dy)) < 8: return i
    return -1

def push_undo():
    undo_stack.append(copy.deepcopy((nodes, edges)))
    redo_stack.clear()

def undo():
    if undo_stack:
        redo_stack.append(copy.deepcopy((nodes, edges)))
        nd, ed = undo_stack.pop()
        nodes.clear(); nodes.extend(nd)
        edges.clear(); edges.extend(ed)
        reset_colors()

def redo():
    if redo_stack:
        undo_stack.append(copy.deepcopy((nodes, edges)))
        nd, ed = redo_stack.pop()
        nodes.clear(); nodes.extend(nd)
        edges.clear(); edges.extend(ed)
        reset_colors()

def reset_colors():
    node_colors.clear(); node_colors.extend([C_NODE_DEF]*len(nodes))
    edge_colors.clear();  edge_colors.extend([C_EDGE_DEF]*len(edges))

def set_msg(*lines):
    msg_lines.clear(); msg_lines.extend(lines)

# ── algo helpers (called from background thread) ─────────────────
def adelay(ms=160):
    time.sleep(ms/1000)

def put_node(i, col): update_q.put(("node", i, col))
def put_edge(i, col): update_q.put(("edge", i, col))
def put_done(*msgs):  update_q.put(("done",)+msgs)

# ── algorithms ───────────────────────────────────────────────────
def algo_dfs(start):
    adj = get_adj()
    vis = [False]*len(nodes)
    put_node(start, C_NODE_SRC); adelay(200)

    def dfs(u):
        vis[u] = True; put_node(u, C_NODE_VIS); adelay(180)
        for v in adj[u]:
            ei = get_edge_index(u, v)
            if not vis[v]:
                if ei >= 0: put_edge(ei, C_EDGE_TREE)
                dfs(v)
            else:
                if ei >= 0: put_edge(ei, C_EDGE_BACK)
        adelay(60)

    dfs(start)
    put_done("DFS complete.",
             "Green = tree edges    Red = back/cross edges.")

def algo_bfs(start):
    adj = get_adj()
    vis = [False]*len(nodes); dis = [-1]*len(nodes)
    bq = q_mod.Queue()
    bq.put(start); vis[start] = True; dis[start] = 0
    put_node(start, C_NODE_SRC); adelay(200)
    while not bq.empty():
        u = bq.get(); put_node(u, C_NODE_VIS); adelay(180)
        for v in adj[u]:
            if not vis[v]:
                vis[v] = True; dis[v] = dis[u]+1
                ei = get_edge_index(u, v)
                if ei >= 0: put_edge(ei, C_EDGE_TREE)
                bq.put(v); adelay(150)
    d_str = "  ".join(f"n{i}:d{dis[i]}" for i in range(len(nodes)) if dis[i] >= 0)
    put_done("BFS complete - distances from source:", d_str)

def algo_dijkstra(start, end):
    adj = get_adj(weighted=True)
    INF = float('inf')
    d = [INF]*len(nodes); prev = [-1]*len(nodes); d[start] = 0
    put_node(start, C_NODE_SRC)
    pq = [(0, start)]; adelay(200)
    while pq:
        dk, u = heapq.heappop(pq)
        if dk > d[u]: continue
        put_node(u, C_NODE_VIS); adelay(150)
        for v, w in adj[u]:
            nd = dk+w
            if nd < d[v]:
                d[v] = nd; prev[v] = u
                ei = get_edge_index(u, v)
                if ei >= 0: put_edge(ei, C_EDGE_TREE)
                heapq.heappush(pq, (nd, v)); adelay(100)
    if end >= 0 and d[end] < INF:
        put_node(end, C_NODE_DST)
        cur = end
        while prev[cur] >= 0:
            ei = get_edge_index(prev[cur], cur)
            if ei >= 0: put_edge(ei, C_EDGE_PATH)
            cur = prev[cur]
        put_node(start, C_NODE_SRC)
        put_done(f"Dijkstra: node {start} to node {end}   distance = {d[end]}",
                 "Yellow path = shortest route.")
    elif end >= 0:
        put_done(f"Node {end} is NOT reachable from node {start}.")
    else:
        info = "  ".join(f"n{i}:{d[i] if d[i]<INF else 'inf'}" for i in range(len(nodes)))
        put_done(f"Dijkstra from node {start}:", info)

def algo_prim():
    if directed:
        put_done("MST requires undirected mode - toggle the switch first."); return
    n = len(nodes)
    in_mst = [False]*n; key = [float('inf')]*n; par = [-1]*n; key[0] = 0
    adj = get_adj(weighted=True); pq = [(0, 0)]; total = 0
    while pq:
        k, u = heapq.heappop(pq)
        if in_mst[u]: continue
        in_mst[u] = True; total += k; put_node(u, C_NODE_VIS)
        if par[u] >= 0:
            ei = get_edge_index(par[u], u)
            if ei >= 0: put_edge(ei, C_EDGE_MST)
        adelay(200)
        for v, w in adj[u]:
            if not in_mst[v] and w < key[v]:
                key[v] = w; par[v] = u; heapq.heappush(pq, (w, v))
    put_done(f"Prim's MST complete - total weight approx {total}",
             "Purple edges form the minimum spanning tree.")

def algo_bridges():
    n = len(nodes); adj = get_adj(); timer = [0]
    disc = [-1]*n; low = [-1]*n; art = set(); brdg = []

    def dfs(u, par):
        disc[u] = low[u] = timer[0]; timer[0] += 1
        put_node(u, C_NODE_VIS); adelay(150); ch = 0
        for v in adj[u]:
            if disc[v] == -1:
                ch += 1; ei = get_edge_index(u, v)
                if ei >= 0: put_edge(ei, C_EDGE_TREE); adelay(120)
                dfs(v, u); low[u] = min(low[u], low[v])
                if low[v] > disc[u]:
                    brdg.append((u, v)); ei = get_edge_index(u, v)
                    if ei >= 0: put_edge(ei, C_EDGE_BRDG)
                if par != -1 and low[v] >= disc[u]: art.add(u)
                if par == -1 and ch > 1: art.add(u)
            elif v != par:
                low[u] = min(low[u], disc[v]); ei = get_edge_index(u, v)
                if ei >= 0: put_edge(ei, C_EDGE_BACK)

    for i in range(n):
        if disc[i] == -1: dfs(i, -1)
    for a in art: put_node(a, C_NODE_ART)
    put_done(f"Found {len(art)} articulation point(s) and {len(brdg)} bridge(s).",
             "Yellow nodes = articulation pts    Blue edges = bridges.")

def algo_cycle():
    n = len(nodes); adj = get_adj(); found = [False]
    if directed:
        col = [0]*n
        def dfs(u):
            col[u] = 1; put_node(u, C_NODE_VIS); adelay(130)
            for v in adj[u]:
                ei = get_edge_index(u, v)
                if col[v] == 1:
                    found[0] = True
                    if ei >= 0: put_edge(ei, C_EDGE_BACK)
                elif col[v] == 0:
                    if ei >= 0: put_edge(ei, C_EDGE_TREE)
                    dfs(v)
            col[u] = 2
        for i in range(n):
            if col[i] == 0: dfs(i)
    else:
        vis = [False]*n
        def dfs2(u, p):
            vis[u] = True; put_node(u, C_NODE_VIS); adelay(130)
            for v in adj[u]:
                ei = get_edge_index(u, v)
                if not vis[v]:
                    if ei >= 0: put_edge(ei, C_EDGE_TREE)
                    dfs2(v, u)
                elif v != p:
                    found[0] = True
                    if ei >= 0: put_edge(ei, C_EDGE_BACK)
        for i in range(n):
            if not vis[i]: dfs2(i, -1)
    if found[0]: put_done("Cycle DETECTED!  Red edges = back-edges that create cycles.")
    else:         put_done("No cycle found in this graph.")

def launch_algo(fn, *args):
    global algo_running
    if algo_running: return
    reset_colors()
    algo_running = True
    def runner():
        global algo_running
        try:    fn(*args)
        finally: algo_running = False
    threading.Thread(target=runner, daemon=True).start()

# ── drawing ──────────────────────────────────────────────────────
def draw_rounded_rect(surf, col, rect, r=8):
    pygame.draw.rect(surf, col, rect, border_radius=r)

def draw_button(x, y, w, h, label, active=False, danger=False, small=False):
    base = (55, 55, 75) if not active else ACCENT
    if danger: base = (160, 45, 45)
    hov = pygame.Rect(x, y, w, h).collidepoint(pygame.mouse.get_pos())
    col = tuple(min(255, c+22) for c in base) if hov else base
    draw_rounded_rect(screen, col, (x, y, w, h))
    f = font_small if small else font_med
    t = f.render(label, True, TEXT_BRIGHT)
    screen.blit(t, (x+w//2-t.get_width()//2, y+h//2-t.get_height()//2))
    return pygame.Rect(x, y, w, h)

def draw_section(x, y, label):
    t = font_tiny.render(label.upper(), True, TEXT_DIM)
    screen.blit(t, (x, y))
    pygame.draw.line(screen, TEXT_DIM, (x, y+t.get_height()+2),
                     (SIDEBAR_W-8, y+t.get_height()+2), 1)

def draw_sidebar():
    pygame.draw.rect(screen, BG_SIDEBAR, (0, 0, SIDEBAR_W, SCREEN_H))
    t = font_big.render("Graph Editor", True, ACCENT)
    screen.blit(t, (SIDEBAR_W//2-t.get_width()//2, 10))
    t2 = font_tiny.render("Algorithm Visualizer", True, TEXT_DIM)
    screen.blit(t2, (SIDEBAR_W//2-t2.get_width()//2, 34))

    y = 56
    draw_section(8, y, "Graph Settings"); y += 20
    tog_lbl = "Mode: Directed ON" if directed else "Mode: Undirected"
    b_tog = draw_button(8, y, SIDEBAR_W-16, 26, tog_lbl, small=True); y += 32
    b_undo = draw_button(8, y, (SIDEBAR_W-20)//2, 26, "Undo", small=True)
    b_redo = draw_button(8+(SIDEBAR_W-20)//2+4, y, (SIDEBAR_W-20)//2, 26, "Redo", small=True); y += 32

    draw_section(8, y, "Edit Graph"); y += 20
    b_an = draw_button(8, y, SIDEBAR_W-16, 28, "Add Node",   active=(state=="add_node")); y += 32
    b_ae = draw_button(8, y, SIDEBAR_W-16, 28, "Add Edge",   active=(state in ("add_edge_a","add_edge_b"))); y += 32
    b_dn = draw_button(8, y, SIDEBAR_W-16, 28, "Delete Node",danger=(state=="del_node"), active=(state=="del_node")); y += 32
    b_de = draw_button(8, y, SIDEBAR_W-16, 28, "Delete Edge",danger=(state=="del_edge"), active=(state=="del_edge")); y += 32
    b_cl = draw_button(8, y, SIDEBAR_W-16, 28, "Clear All",  danger=True); y += 36

    draw_section(8, y, "Algorithms"); y += 20
    b_dfs = draw_button(8, y, SIDEBAR_W-16, 26, "DFS",            small=True); y += 30
    b_bfs = draw_button(8, y, SIDEBAR_W-16, 26, "BFS",            small=True); y += 30
    b_dij = draw_button(8, y, SIDEBAR_W-16, 26, "Dijkstra SSSP",  small=True); y += 30
    b_mst = draw_button(8, y, SIDEBAR_W-16, 26, "Prim's MST",     small=True); y += 30
    b_brg = draw_button(8, y, SIDEBAR_W-16, 26, "Bridges & APs",  small=True); y += 30
    b_cyc = draw_button(8, y, SIDEBAR_W-16, 26, "Cycle Detect",   small=True); y += 30
    b_rst = draw_button(8, y, SIDEBAR_W-16, 26, "Reset Colors",   small=True); y += 34

    draw_section(8, y, "Info"); y += 18
    for line in [f"Nodes: {len(nodes)}", f"Edges: {len(edges)}",
                 f"Type: {'Directed' if directed else 'Undirected'}"]:
        t = font_small.render(line, True, TEXT_MAIN)
        screen.blit(t, (10, y)); y += 16

    if algo_running:
        t = font_small.render("Running...", True, C_NODE_ART)
        screen.blit(t, (SIDEBAR_W//2-t.get_width()//2, y+4))

    return dict(tog=b_tog, undo=b_undo, redo=b_redo,
                an=b_an, ae=b_ae, dn=b_dn, de=b_de, cl=b_cl,
                dfs=b_dfs, bfs=b_bfs, dij=b_dij, mst=b_mst,
                brg=b_brg, cyc=b_cyc, rst=b_rst)

def draw_canvas():
    pygame.draw.rect(screen, BG_CANVAS, (CANVAS_X, 0, CANVAS_W, SCREEN_H-INFOPANEL_H))
    for gx in range(CANVAS_X, SCREEN_W, 40):
        pygame.draw.line(screen, (44, 44, 56), (gx, 0), (gx, SCREEN_H-INFOPANEL_H), 1)
    for gy in range(0, SCREEN_H-INFOPANEL_H, 40):
        pygame.draw.line(screen, (44, 44, 56), (CANVAS_X, gy), (SCREEN_W, gy), 1)

def draw_edges():
    for i, e in enumerate(edges):
        ax, ay = node_center(e[0]); bx, by = node_center(e[1])
        col = edge_colors[i] if i < len(edge_colors) else C_EDGE_DEF
        w = 3 if col != C_EDGE_DEF else 2
        pygame.draw.line(screen, col, (ax, ay), (bx, by), w)

        if directed:
            ang = math.atan2(by-ay, bx-ax)
            tx = bx-int(NODE_R*math.cos(ang)); ty = by-int(NODE_R*math.sin(ang))
            l = (tx+int(11*math.cos(ang+2.5)), ty+int(11*math.sin(ang+2.5)))
            r = (tx+int(11*math.cos(ang-2.5)), ty+int(11*math.sin(ang-2.5)))
            pygame.draw.polygon(screen, col, [l, (tx, ty), r])

def draw_nodes():
    for i, n in enumerate(nodes):
        col = node_colors[i] if i < len(node_colors) else C_NODE_DEF
        gs = pygame.Surface((NODE_R*4, NODE_R*4), pygame.SRCALPHA)
        pygame.draw.circle(gs, (*col, 35), (NODE_R*2, NODE_R*2), NODE_R+9)
        screen.blit(gs, (n[0]-NODE_R*2, n[1]-NODE_R*2))
        pygame.draw.circle(screen, col, (n[0], n[1]), NODE_R)
        border = (255, 255, 255) if i == selected_a else (15, 15, 25)
        pygame.draw.circle(screen, border, (n[0], n[1]), NODE_R, 2)
        lbl = font_small.render(str(i), True, BG_DARK)
        screen.blit(lbl, (n[0]-lbl.get_width()//2, n[1]-lbl.get_height()//2))
    if state == "add_edge_b" and selected_a >= 0:
        mx2, my2 = pygame.mouse.get_pos()
        ax, ay = node_center(selected_a)
        pygame.draw.line(screen, (150, 150, 190), (ax, ay), (mx2, my2), 2)

def draw_info():
    py = SCREEN_H-INFOPANEL_H
    pygame.draw.rect(screen, BG_INFO, (CANVAS_X, py, CANVAS_W, INFOPANEL_H))
    pygame.draw.line(screen, ACCENT, (CANVAS_X, py), (SCREEN_W, py), 2)
    x0 = CANVAS_X+12
    for i, line in enumerate(msg_lines[-4:]):
        col = TEXT_MAIN if i == len(msg_lines[-4:])-1 else TEXT_DIM
        t = font_small.render(line, True, col); screen.blit(t, (x0, py+8+i*20))
    hints = {
        "idle":       "Ready",
        "add_node":   "Click canvas to place a node  (ESC to stop)",
        "add_edge_a": "Click the SOURCE node",
        "add_edge_b": "Click the TARGET node  (ESC to cancel)",
        "del_node":   "Click a node to delete it",
        "del_edge":   "Click near an edge to delete it",
        "choose_src": "Click a node as algorithm SOURCE",
        "choose_dst": "Click a node as DESTINATION",
        "result":     "Done - press R to reset colours",
    }
    hint = hints.get(state, state)
    t = font_med.render(f"  {hint}", True, ACCENT)
    screen.blit(t, (x0, py+INFOPANEL_H-28))

def draw_all():
    screen.fill(BG_DARK)
    draw_canvas()
    btns = draw_sidebar()
    draw_edges()
    draw_nodes()
    draw_info()
    pygame.display.flip()
    return btns

# ── main loop ────────────────────────────────────────────────────
reset_colors()
running = True

while running:
    # drain algo update queue (thread-safe colour updates)
    try:
        while True:
            item = update_q.get_nowait()
            kind = item[0]
            if kind == "node" and item[1] < len(node_colors):
                node_colors[item[1]] = item[2]
            elif kind == "edge" and item[1] < len(edge_colors):
                edge_colors[item[1]] = item[2]
            elif kind == "done":
                set_msg(*item[1:]); state = "result"
    except q_mod.Empty:
        pass

    btns = draw_all()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        elif event.type == pygame.KEYDOWN:
            k = event.key
            if k == pygame.K_ESCAPE:
                state = "idle"; selected_a = -1; set_msg("Cancelled.")
            elif k == pygame.K_r:
                reset_colors(); state = "idle"; set_msg("Colors reset.")
            elif k == pygame.K_z and event.mod & pygame.KMOD_CTRL:
                undo(); set_msg("Undo.")
            elif k == pygame.K_y and event.mod & pygame.KMOD_CTRL:
                redo(); set_msg("Redo.")
            elif k == pygame.K_n and not algo_running:
                state = "add_node"; set_msg("Click canvas to place nodes.")
            elif k == pygame.K_e and not algo_running:
                state = "add_edge_a"; selected_a = -1; set_msg("Click source node.")
            elif k == pygame.K_d and not algo_running:
                directed = not directed; reset_colors()
                set_msg(f"Switched to {'Directed' if directed else 'Undirected'} mode.")

        elif event.type == pygame.MOUSEBUTTONUP and not algo_running:
            mx, my = event.pos

            # ── sidebar clicks ──
            if mx < SIDEBAR_W:
                if btns["tog"].collidepoint(mx, my):
                    directed = not directed; reset_colors()
                    set_msg(f"Switched to {'Directed' if directed else 'Undirected'} mode.")
                elif btns["undo"].collidepoint(mx, my): undo(); set_msg("Undo.")
                elif btns["redo"].collidepoint(mx, my): redo(); set_msg("Redo.")
                elif btns["an"].collidepoint(mx, my):
                    state = "add_node"; set_msg("Click canvas to place nodes.")
                elif btns["ae"].collidepoint(mx, my):
                    state = "add_edge_a"; selected_a = -1; set_msg("Click source node.")
                elif btns["dn"].collidepoint(mx, my):
                    state = "del_node"; set_msg("Click a node to delete.")
                elif btns["de"].collidepoint(mx, my):
                    state = "del_edge"; set_msg("Click near an edge to delete.")
                elif btns["cl"].collidepoint(mx, my):
                    push_undo(); nodes.clear(); edges.clear(); reset_colors()
                    state = "idle"; set_msg("Canvas cleared.")
                elif btns["dfs"].collidepoint(mx, my):
                    if not nodes: set_msg("Add nodes first!")
                    else: algo_pending = "dfs"; state = "choose_src"; set_msg("DFS: click a source node.")
                elif btns["bfs"].collidepoint(mx, my):
                    if not nodes: set_msg("Add nodes first!")
                    else: algo_pending = "bfs"; state = "choose_src"; set_msg("BFS: click a source node.")
                elif btns["dij"].collidepoint(mx, my):
                    if not nodes: set_msg("Add nodes first!")
                    else: algo_pending = "dij"; state = "choose_src"; set_msg("Dijkstra: click SOURCE node.")
                elif btns["mst"].collidepoint(mx, my):
                    if not nodes: set_msg("Add nodes first!")
                    else: launch_algo(algo_prim); state = "idle"; set_msg("Prim's MST running...")
                elif btns["brg"].collidepoint(mx, my):
                    if not nodes: set_msg("Add nodes first!")
                    else: launch_algo(algo_bridges); state = "idle"; set_msg("Finding bridges & APs...")
                elif btns["cyc"].collidepoint(mx, my):
                    if not nodes: set_msg("Add nodes first!")
                    else: launch_algo(algo_cycle); state = "idle"; set_msg("Cycle detection running...")
                elif btns["rst"].collidepoint(mx, my):
                    reset_colors(); state = "idle"; set_msg("Colors reset.")

            # ── canvas clicks ──
            else:
                if my >= SCREEN_H-INFOPANEL_H: continue

                if state == "add_node":
                    push_undo()
                    nodes.append([mx, my]); node_colors.append(C_NODE_DEF)
                    edge_colors.clear(); edge_colors.extend([C_EDGE_DEF]*len(edges))
                    set_msg(f"Node {len(nodes)-1} placed at ({mx},{my}).")

                elif state == "add_edge_a":
                    ni = node_at(mx, my)
                    if ni >= 0: selected_a = ni; state = "add_edge_b"; set_msg(f"Source = node {ni}.  Click target.")
                    else: set_msg("Click ON a node.")

                elif state == "add_edge_b":
                    ni = node_at(mx, my)
                    if ni >= 0 and ni != selected_a:
                        if edge_exists(selected_a, ni): set_msg("Edge already exists.")
                        else:
                            push_undo(); w = edist(selected_a, ni)
                            edges.append([selected_a, ni, w]); edge_colors.append(C_EDGE_DEF)
                            set_msg(f"Edge {selected_a} to {ni} added (weight {w}).")
                        selected_a = -1; state = "add_edge_a"
                    elif ni == selected_a: set_msg("Self-loops not supported.")
                    else: set_msg("No node there.")

                elif state == "del_node":
                    ni = node_at(mx, my)
                    if ni >= 0:
                        push_undo()
                        edges_new = [[a-int(a>ni), b-int(b>ni), w]
                                     for a, b, w in edges if a != ni and b != ni]
                        edges.clear(); edges.extend(edges_new)
                        nodes.pop(ni); reset_colors(); set_msg(f"Node {ni} deleted.")
                    else: set_msg("No node there.")

                elif state == "del_edge":
                    ei = edge_at(mx, my)
                    if ei >= 0:
                        push_undo(); e = edges.pop(ei)
                        if ei < len(edge_colors): edge_colors.pop(ei)
                        set_msg(f"Edge {e[0]} to {e[1]} deleted.")
                    else: set_msg("Click closer to an edge line.")

                elif state == "choose_src":
                    ni = node_at(mx, my)
                    if ni >= 0:
                        src_node = ni
                        if algo_pending == "dij":
                            state = "choose_dst"; set_msg(f"Source = node {ni}.  Click DESTINATION.")
                        else:
                            fn = {"dfs": algo_dfs, "bfs": algo_bfs}[algo_pending]
                            launch_algo(fn, src_node); state = "idle"
                            set_msg(f"{algo_pending.upper()} running from node {ni}...")
                    else: set_msg("Click ON a node.")

                elif state == "choose_dst":
                    ni = node_at(mx, my)
                    if ni >= 0:
                        dst_node = ni; launch_algo(algo_dijkstra, src_node, dst_node)
                        state = "idle"; set_msg(f"Dijkstra: node {src_node} to node {ni}...")
                    else: set_msg("Click ON a node for destination.")

    clock.tick(60)

pygame.quit()